#**Approche avec les équations de Bellman**

##**7. Exemple**

Regardons maintenant un exemple simple afin de synthétiser l'ensemble des concepts que nous avons vu : valeur d'action, valeur d'état et équation de Bellman.

####**7.1 Problématique**

On considère un environnement composé de 3 états A, B et C. L'agent commence sur l'état A. Le but de l'agent est d'atteindre la cible C. L'état B est un piège. L'agent peut choisir parmi 4 actions mais pour simplifier les notations on les considère comme ceci :
- 0 : Gauche ou Droite
- 1 : Haut ou bas

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL16.png?raw=true" width="240"></img></center>

La **fonction valeur d'action** dans ce cas peut être exprimée sous forme d'un tableau qui fait correspondre les valeurs de chaque paire état-action. Au début, celle-ci est initialisée à 0 :

<table>
<thead><tr>
<th style="text-align:center">État: S</th>
<th style="text-align:center">Action: A</th>
<th style="text-align:center">Valeur</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">0</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">1</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td style="text-align:center">B</td>
<td style="text-align:center">0</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td style="text-align:center">B</td>
<td style="text-align:center">1</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td style="text-align:center">C</td>
<td style="text-align:center">0</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td style="text-align:center">C</td>
<td style="text-align:center">1</td>
<td style="text-align:center">0</td>
</tr>
</tbody>
</table>

Pour mettre en place l'apprentissage, il faut également définir une table des transitions ainsi que les récompenses. Prenons par exemple la table des transitions suivantes (pour simplifier, on considère que les probabilités de transition et les récompenses sont identiques pour chaque état). La table ci-dessous ne montre que ce qui concerne l'état A et B mais on suivra la même idée pour l'état C:

<table>
<thead><tr>
<th style="text-align:center">État: S</th>
<th style="text-align:center">Action: A</th>
<th style="text-align:center">État suivant: S'</th>
<th style="text-align:center">Probabilité de transition: P</th>
<th style="text-align:center">Récompense: R</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">0</td>
<td style="text-align:center">A</td>
<td style="text-align:center">0.2</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">0</td>
<td style="text-align:center">B</td>
<td style="text-align:center">0.5</td>
<td style="text-align:center">-1</td>
</tr>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">0</td>
<td style="text-align:center">C</td>
<td style="text-align:center">0.3</td>
<td style="text-align:center">1</td>
</tr>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">1</td>
<td style="text-align:center">A</td>
<td style="text-align:center">0.3</td>
<td style="text-align:center">0</td>
</tr>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">1</td>
<td style="text-align:center">B</td>
<td style="text-align:center">0.2</td>
<td style="text-align:center">-1</td>
</tr>
<tr>
<td style="text-align:center">A</td>
<td style="text-align:center">1</td>
<td style="text-align:center">C</td>
<td style="text-align:center">0.5</td>
<td style="text-align:center">1</td>
</tr>
</tbody>
</table>

####**7.2 Programme Python**

**Définition de la table des transitions**

La fonction suivante permet de construire la table des transitions. Pour chaque état de l'environnement et pour chaque action pouvant être effectuée sur cet état, la table contient les informations suivantes:

- L'index de l'états suivant sur lequel arrive l'agent après avoir sélectionné une action donnée sur un état donné 
- La probabilité de transition d'arriver sur cet état suivant
- La récompense obtenue suite à cette transition

Nous avons donc besoin de définir un tenseur qui pourra contenir les trois informations précédentes (index de l'état suivant, probabilité de transition et récompense) pour chaque paire (état, action).

L'idée est donc de fusionner trois tenseurs:
- Tenseur n°1 pour accéder aux états suivants : (nbr_etat,nbr_actions,nbr_etats_suivants)
- Tenseur n°2 pour accéder aux probabilités : (nbr_etat,nbr_actions,nbr_etats_suivants)
- Tenseur n°3 pour accéder aux récompenses : (nbr_etat,nbr_actions,nbr_etats_suivants)

Pour faire cela, on peut par exemple ajouter une dimension sur l'axe 0 et définir un tenseur au format :

$\quad$(3,nbr_etat,nbr_actions,nbr_etats_suivants)

Sachant que:
- l'indice 0 de l'axe 0 ciblera les états suivants
- l'indice 1 de l'axe 0 ciblera les probabilités
- l'indice 2 de l'axe 0 ciblera les récompenses




Ici, nous allons plutôt construire un tenseur numpy avec un type personnalisé. Cela rendra la manipulation des données plus intuitive.

In [ ]:
import numpy as np

nombre_etats = 3
nombre_actions = 2

#####################################################
# Construction de la table des transitions
# Format :  (nbr_etats, nbr_actions, nbr_etat_suivant)['etat_suivant' | 'proba' | 'recompense']
#############################################################################
def ConstructionTableTransitions(nbr_etats, nbr_actions, nbr_etat_suivants):
  # Structure de la table de transition
  structure_table = np.dtype([('etat_suivant', np.uint16),
                            ('proba', np.float32),
                            ('recompense', np.int8)])

  # Initialisation de la table des transitions à 0
  table_transition = np.zeros((nbr_etats, nbr_actions, nbr_etat_suivants),
                              dtype=structure_table)

  # Définition des états suivants
  # A=0 ; B=1 et C=2
  for etat in range(0,nbr_etats):
    for action in range (0,nbr_actions):
      table_transition[etat,action,0]['etat_suivant'] = 0
      table_transition[etat,action,1]['etat_suivant'] = 1
      table_transition[etat,action,2]['etat_suivant'] = 2
  
  # Définition des probabilités
  # Etat A et B
  for etat in range(0,2):
    table_transition[etat,0,0]['proba'] = 0.2       # A->A (action=0)
    table_transition[etat,0,1]['proba'] = 0.5       # A->B (action=0)
    table_transition[etat,0,2]['proba'] = 0.3       # A->C (action=0)

    table_transition[etat,1,0]['proba'] = 0.3       # A->A (action=1)
    table_transition[etat,1,1]['proba'] = 0.2       # A->B (action=1)
    table_transition[etat,1,2]['proba'] = 0.5       # A->C (action=1)

  # Etat C
  table_transition[2,0,0]['proba'] = 0.3            # C->A (action=0)
  table_transition[2,0,1]['proba'] = 0.5            # C->B (action=0)
  table_transition[2,0,2]['proba'] = 0.2            # C->C (action=0)

  table_transition[2,1,0]['proba'] = 0.5            # C->A (action=1)
  table_transition[2,1,1]['proba'] = 0.3            # C->B (action=1)
  table_transition[2,1,2]['proba'] = 0.2            # C->C (action=1)

  # Définition des récompenses
  for etat in range(0,nbr_etats):
    for action in range (0,nbr_actions):
      for etat_suivant in range (0,nbr_etat_suivants):
        # Si l'agent tombe sur un piège alors recompense = -1
        if table_transition[etat,action,etat_suivant]['etat_suivant'] == 1:
          table_transition[etat,action,etat_suivant]['recompense'] = -1

        # Si l'agent tombe sur la cible alors recompense = 1 (sauf si piege)
        elif table_transition[etat,action,etat_suivant]['etat_suivant'] == 2:
          table_transition[etat,action,etat_suivant]['recompense'] = 1
    
  return table_transition

In [ ]:
import numpy as np
import pandas as pd

nombre_etats = 3
nombre_actions = 2

# Construction de la table des transitions
table_transition = ConstructionTableTransitions(nombre_etats,nombre_actions,nombre_etats)

# Converison de la table au format Pandas
data = []
columns = ['Etat', 'Action', 'Etat suivant', 'Proba', 'Récompense']
for index_etat in range(0,nombre_etats):
  for action in range(0,nombre_actions):
    for etat_suivant in range(0,nombre_etats):
      data.append([index_etat, action,
                   table_transition[index_etat,action,etat_suivant]['etat_suivant'],
                   table_transition[index_etat,action,etat_suivant]['proba'],
                   table_transition[index_etat,action,etat_suivant]['recompense']])
df_table = pd.DataFrame(data=data,columns=columns)

# Affiche la table
print(df_table.to_string(index=False))

####**Application de l'équation de Bellman**

Commençons par appliquer l'équation de Bellman afin de calculer la valeur de l'action 0 lorsque l'agent se trouve sur l'état 0:

In [ ]:
# Equation de Bellman
# Q(s,a) = proba * (recompense + gamma * valeur_etat_suivant)

gamma = 1

# Initialisation de la table des valeurs d'état
V = np.zeros((nombre_etats))         # (nombre_etats)

# Initialisation de la table des valeurs d'action
Q_table = np.zeros((nombre_etats,nombre_actions))         # (nombre_etats,nombre_actions)

# Calcul de la valeur d'action de l'état A en suivant l'action 0
# La valeur est calculé en sommant toutes les possibilités offertes
# à l'agent qui suive le fait de prendre l'action 0 sur l'état A
for etat_suivant in range(0,nombre_etats):
  # Récupère la probabilité et la récompense
  proba = table_transition[0,0,etat_suivant]['proba']
  recompense = table_transition[0,0,etat_suivant]['recompense']

  # Equation de Bellman
  Q_table[0,0] = Q_table[0,0] + proba*(recompense + gamma*V[etat_suivant])

print(Q_table[0,0])

On peut faire de même si c'est l'action 1 qui est prise sur l'état A:

In [ ]:
# Calcul de la valeur d'action de l'état A en suivant l'action 1
for etat_suivant in range(0,nombre_etats):
  # Récupère la probabilité et la récompense
  proba = table_transition[0,1,etat_suivant]['proba']
  recompense = table_transition[0,1,etat_suivant]['recompense']

  # Equation de Bellman
  Q_table[0,1] = Q_table[0,1] + proba*(recompense + gamma*V[etat_suivant])
print(Q_table[0,1])

Regardons à quoi ressemble alors la table des valeurs d'actions :

In [ ]:
# Converision de la table des valeurs d'action au format Pandas
data = []
columns = ['Etat', 'Action', 'Valeur']
for index_etat in range(0,nombre_etats):
  for action in range(0,nombre_actions):
    data.append([index_etat, action, Q_table[index_etat,action]])
df_Qtable = pd.DataFrame(data=data,columns=columns)

# Affiche la table
print("Table des valeurs d'action:\n")
print(df_Qtable.to_string(index=False))

**Durant l'apprentissage, l'agent suit la politique la plus optimisée possible, c'est-à-dire celle qui remporte le plus haut revenu. La règle pour l'agent est donc d'utiliser l'action avec la plus haute valeur. Dans ce cas, si l'agent se trouve sur l'état A, il suivra l'action 1.**

La table des valeurs d'état est mise à jour en utilisant la plus haute valeur de la table des actions parmi toutes les actions possible sur l'état concerné:

In [ ]:
V[0] = np.amax(Q_table[0,:])
print(V)

Après cette première itération, la table des états ressemble donc à celle-ci:

<table>
<thead><tr>
<th>État</th>
<th>Valeur</th>
</tr>
</thead>
<tbody>
<tr>
<td>A</td>
<td>0.3</td>
</tr>
<tr>
<td>B</td>
<td>0</td>
</tr>
<tr>
<td>C</td>
<td>0</td>
</tr>
</tbody>
</table>

L'agent suit la même procédure pour l'ensemble des états et répète le jeu:

In [ ]:
# Equation de Bellman
# Q(s,a) = proba * (recompense + gamma * valeur_etat_suivant)

gamma = 1

# Initialisation de la table des valeurs d'état
V = np.zeros((nombre_etats))         # (nombre_etats)

# Initialisation de la table des valeurs d'action
Q_table = np.zeros((nombre_etats,nombre_actions))         # (nombre_etats,nombre_actions)

print("V::",V)
for i in range(5):
  print("Épisode ... :",i)
  for etat in range(0,nombre_etats):
    for action in range(0,nombre_actions):
      for etat_suivant in range(0,nombre_etats):
        # Récupère la probabilité et la récompense
        proba = table_transition[etat,action,etat_suivant]['proba']
        recompense = table_transition[etat,action,etat_suivant]['recompense']

        # Equation de Bellman
        Q_table[etat,action] = Q_table[etat,action] + proba*(recompense + gamma*V[etat_suivant])
      
    # Mise à jour de la table des valeurs d'états
    V[etat] = np.amax(Q_table[etat,:])
  print("V::",V)

In [ ]:
# Conversion de la table des valeurs d'action au format Pandas
data = []
columns = ['Etat', 'Action', 'Valeur']
for index_etat in range(0,nombre_etats):
  for action in range(0,nombre_actions):
    data.append([index_etat, action, Q_table[index_etat,action]])
df_Qtable = pd.DataFrame(data=data,columns=columns)

# Affiche la table des valeurs d'actions
print("Table des valeurs d'action:\n")
print(df_Qtable.to_string(index=False))

L'agent, lorsqu'il part de l'état A (0) va donc prendre l'action 1 (haut-bas) et donc se diriger vers l'état C ce qui est logique car cela le dirige vers l'état C:

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL16.png?raw=true" width="240"></img></center>

In [ ]:
# Conversion de la table des valeurs d'état au format Pandas
data = []
columns = ['Etat', 'Valeur']
for index_etat in range(0,nombre_etats):
  data.append([index_etat, V[index_etat]])
df_Vtable = pd.DataFrame(data=data,columns=columns)

# Affiche la table des valeurs d'actions
print("Table des valeurs d'état:\n")
print(df_Vtable.to_string(index=False))

Il en de même pour les valeurs d'état : L'agent se dirige vers l'état C (2) cet état possède la plus haute valeur.